Introductie

P1

P2

P3

P4

## De invloed van klimaat en geografie op voedselproductie

Niet elk land beschikt over dezelfde natuurlijke omstandigheden om voedsel te produceren. Factoren zoals neerslag, temperatuur en klimaatzones bepalen welke gewassen efficiënt kunnen groeien. In landen met een ongunstig klimaat moeten gewassen onder intensievere en vaak minder duurzame omstandigheden worden verbouwd.
Daarnaast speelt de geografische ligging een rol in hoe voedsel wordt verhandeld: landen die ver van exportmarkten liggen, hebben hogere transportkosten en extra uitstoot, wat hun voedselsysteem minder duurzaam maakt.
In dit perspectief onderzoeken we hoe deze geografische factoren — natuurlijke omstandigheden en ligging ten opzichte van wereldmarkten — bijdragen aan verschillen in de duurzaamheid van nationale voedselsystemen.

#### Klimaatzones en natuurlijke omstandigheden beïnvloeden duurzaamheid
Landen verschillen sterk in klimaat. Tropische regio’s kennen bijvoorbeeld hoge temperaturen en regelmatige regenval, terwijl droge of koude gebieden kampen met beperkte groeiseizoenen en lagere landbouwopbrengsten per hectare.
In warmere, vruchtbare klimaatzones kunnen gewassen natuurlijk groeien, met weinig extra input. In koudere of drogere gebieden zijn irrigatie, verwarming, kunstmest of import van hulpbronnen nodig om dezelfde opbrengst te halen. Deze extra inputs verhogen de uitstoot per kilogram geproduceerd voedsel.
Daarom is het voedselsysteem in sommige landen per definitie minder duurzaam, simpelweg door hun ligging in een ongunstige klimaatzone.

In [ ]:
import pandas as pd
import plotly.express as px

# Laad temperatuurdata
temp_df = pd.read_csv("datasets/GlobalLandTemperaturesByCountry.csv")
temp_df['dt'] = pd.to_datetime(temp_df['dt'])
temp_df['Year'] = temp_df['dt'].dt.year
landen = ["Netherlands", "Ukraine", "Nigeria", "South Africa", "India", "Japan", "United States", "Brazil", "Indonesia", "Mexico"]
temp_df = temp_df[temp_df['Country'].isin(landen)]
temp_avg = temp_df.groupby(['Country', 'Year'])['AverageTemperature'].mean().reset_index()
temp_avg = temp_avg[(temp_avg['Year'] >= 1960) & (temp_avg['Year'] <= 2020)]

# Laad neerslagdata
precip_df = pd.read_csv("datasets/API_AG.LND.PRCP.MM_DS2_en_csv_v2_4487.csv", skiprows=4)
precip_df = precip_df[precip_df['Country Name'].isin(landen)]
precip_df = precip_df.melt(id_vars=['Country Name'], var_name='Year', value_name='Precipitation')
precip_df.rename(columns={'Country Name': 'Country'}, inplace=True)
precip_df['Year'] = pd.to_numeric(precip_df['Year'], errors='coerce')
precip_df = precip_df.dropna(subset=['Year'])
precip_df = precip_df[(precip_df['Year'] >= 1960) & (precip_df['Year'] <= 2020)]

# Combineer datasets
merged_df = pd.merge(temp_avg, precip_df, on=['Country', 'Year'])

# Maak de interactieve plot
fig = px.line(
    merged_df,
    x="AverageTemperature",
    y="Precipitation",
    color="Country",
    animation_frame="Year",
    title="Gemiddelde temperatuur vs. neerslag (1960–2020)",
    labels={
        "AverageTemperature": "Gemiddelde temperatuur (°C)",
        "Precipitation": "Jaarlijkse neerslag (mm)",
        "Country": "Land"
    }
)

fig.update_layout(legend_title="Land")
fig.show()
